# Introduction

This notebook contains an implementation of Longstaff

NOTE: Clean up unnecessary modules.

In [37]:
import random
import time
import statistics as st
from scipy.stats import norm #For The Anderson-Darling Test
from scipy.stats import expon

from sympy import sieve #For the Halton Sequence

#importing packages
import pandas as pd
import numpy as np

#For regression purposes
from numpy.polynomial.laguerre import lagfit
from numpy.polynomial.laguerre import lagval
from numpy.polynomial.polynomial import polyfit
from numpy.polynomial.polynomial import polyval
from scipy.optimize import curve_fit

#For stochastic optimization purposes
#from noisyopt import minimizeCompass

import datetime as dt
#import pandas_datareader as pdr
import seaborn as sns
import matplotlib.pyplot as plt
import bs4 as bs
import requests
from IPython.display import clear_output
from scipy.stats import mstats
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import RandomizedSearchCV, validation_curve, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
from sklearn.model_selection import GridSearchCV
sns.set()

from statsmodels.base.model import GenericLikelihoodModel

#Functions from other notebooks
#from ipynb.fs.defs.stock_data_suite import TD_stock_data

In [ ]:
random.seed(dt.datetime.now())

# Simulating Brownian Motion

We make use of np.random.normal to simulate the normal random variables we need for the Brownian motion. simulateBrownian and simulateBrownianMultiple are effectively the same thing except the data structure of simulateBrownianMultiple happens to be "nicer" for the LSM code.

The primary purpose for this code is in fact to simulate *geometric* Brownian motion. Let's suppose we want to simulate a stochastic process with dynamics given by

$$dX_t = X_t(\mu\,dt + \sigma\,dW_t)$$

We can do this by simulating

$$dU_t = \Big(\mu - \frac{1}{2}\sigma^2\Big)\,dt + \sigma\,dW_t$$

and then putting $X_t = exp(U_t)$.

In [5]:
def simulateBrownian(mu, sigma, T, time_steps, paths, antithetic = False):
    #The antithetic option is helpful for the American put code
    
    delta = T/time_steps
    
    if antithetic == False:
        increment_grid = mu*delta + np.sqrt(delta)*sigma*np.random.normal(size=(time_steps,paths))
    else:
        random_part = np.sqrt(delta)*sigma*np.random.normal(size=(time_steps,int(paths/2)))
        increment_grid_1 = mu*delta + random_part
        increment_grid_2 = mu*delta - random_part
        increment_grid = np.append(increment_grid_1,increment_grid_2, axis = 1)
    
    #Need to check to make sure the dimensions are correct
    #print(np.shape(increment_grid))
    
    #print("Shape of increment grid:" + str(np.shape(increment_grid))) 
    #print("Shape of zero_vector: " + str(np.shape(zero_vector)))
    output_grid = np.zeros((time_steps,paths))
    
    for i in range(time_steps):
        addition_grid = np.asarray([increment_grid[i,:] if j>=i else [0 for k in range(paths)] for j in range(time_steps)])
        #print(addition_grid)
        #print("Shape of addition grid (" + str(i+1) + "/" + str(time_steps) + "): " + str(np.shape(addition_grid)))
        output_grid = output_grid + addition_grid
    return output_grid

In [43]:
def simulateBrownianMultiple(mu, sigma, T, time_steps, paths, simulations, antithetic = False):
    #The antithetic option is helpful for the American put code
    
    delta = T/time_steps
    
    #prices = np.split(prices,simulations,axis=1)
    
    if antithetic == False:
        increment_grid = mu*delta + np.sqrt(delta)*sigma*np.asarray(norm.rvs(size=time_steps*paths*simulations)).reshape(simulations,time_steps,paths)
    else:
        random_part = np.sqrt(delta)*sigma*np.asarray(norm.rvs(size=time_steps*np.int(paths/2)*simulations)).reshape(simulations,time_steps,np.int(paths/2))
        increment_grid_1 = mu*delta + random_part
        increment_grid_2 = mu*delta - random_part
        increment_grid = np.append(increment_grid_1,increment_grid_2, axis = 2)
    
    #Need to check to make sure the dimensions are correct
    #print(increment_grid)
    
    #The below stuff needs to be rewritten in order for this code to work, probably gonna be messy af LMAO
    
    #print("Shape of increment grid:" + str(np.shape(increment_grid))) 

    #print("Shape of zero_vector: " + str(np.shape(zero_vector)))
    output_grid = np.zeros((simulations,time_steps,paths))
    
    for i in range(time_steps):
        #The addition grid is the increment grid but the first "i" columns (2nd index) are all set to zeros
        
        addition_grid = np.zeros((simulations,time_steps,paths))
        addition_grid = addition_grid + increment_grid[:,i:(i+1),:] #Abusing numpy's broadcasting feature
        addition_grid[:,0:i,:] = 0
        
        #addition_grid = np.asarray([[increment_grid[h,i,:] if j>=i else [0 for k in range(paths)] for j in range(time_steps)] for h in range(simulations)])
        
        #print(addition_grid)
        #print("Shape of addition grid (" + str(i+1) + "/" + str(time_steps) + "): " + str(np.shape(addition_grid)))
        output_grid = output_grid + addition_grid
    return output_grid

# Longstaff \& Schwartz Algorithm

The method provided here is based off of Longstaff \& Schwartz. The reader is referred to the below for more details: https://people.math.ethz.ch/~hjfurrer/teaching/LongstaffSchwartzAmericanOptionsLeastSquareMonteCarlo.pdf

In addition to their work in the paper, some additional functions have been added:
1. Support for call options
2. A setting which assumes the stock price has 


Notes on function arguments: 
1. If graphs == True, then show graphs of the simulated returns and the regression line at each time. This is helpful to make sure the code is performing correctly.
2. If decision_line == True, then show a graph of the optimal exercise strategy as time evolves. Since the method is a Monte Carlo method, the line might look "shaky" when it ought to be smooth.
3. If Prob_ITM == True, then show the CDF of the value of the option at expiration. Necessarily, this includes giving the probability that the option expires in the money.


In [47]:
def LSM(T, K , r, sigma, S_0, time_steps, paths, simulations, 
        prices = np.array([]),
        set_prices = True,
        decision_line = False, 
        early_termination = True, 
        Prob_ITM = False, 
        runtime = False,
        support = (float("-inf"),float("inf")),
        option_type = "put",
        div_times = [],
        div_payouts = [],
        graphs = False):
    
    ###############################################################
    #First, simulate the stock price under the risk-neutral measure
    #Here, I generate "simulations" number of price sets, each with "paths" number of paths.
    if set_prices == True:
        prices = S_0*np.exp(simulateBrownian(r - 0.5*np.power(sigma,2.0), sigma, T, time_steps, paths*simulations, antithetic=True))
    #The scenario of setting prices = False to begin with is useful if we already have a simulated price process to begin with.
    #This is useful for saving computation time when computing implied volatility (see the implied volatility code).
    
    
    #If there are dividend payouts, modify the price process by the dividend payouts
    decision_steps = np.ceil(time_steps*(np.array(div_times)/T)).astype(int) - 1
    #print(decision_steps)
    
    for i in range(len(div_times)):
        j = decision_steps[i]
        jump_amount = np.minimum(np.maximum(prices[j:,:] - support[0],0),div_payouts[i])
        #print(jump_amount)
        prices[j:,:] = prices[j:,:] - jump_amount
    
    #print("Supports: " + str(support))
    
    #If the price has supports, modify the above process so it fits within these supports
    prices = prices.flatten()
    while len(np.where((prices > support[1]) | (prices < support[0]))[0]) > 0:
        prices = np.piecewise(prices, [(prices <= support[1]) & (prices >= support[0]), prices > support[1], prices < support[0]], [lambda x: x, lambda x: 2*support[1] - x, lambda x:2*support[0] - x])
    
    #print("Number of nonpositive prices: "+ str(len(np.where(prices <= 0)[0])))
    
    #This is not efficient but it works. I'll deal with speed later
    prices = prices.reshape(time_steps, paths*simulations)
    
    #print("Price set after reshaping: " + str(prices))
    
    prices = np.array(np.split(prices,simulations,axis=1))
    simulations,time_steps,paths = np.shape(prices)
    #print(prices[0,:,0])
    
    ###########################################
    #Setting up additional parameters for usage

    #Setup for subplots
    if graphs == True:
        fig, axs = plt.subplots(time_steps-1, simulations)
        fig.set_size_inches(6*simulations, 4*time_steps)
        fig.suptitle('Put Option Exercise Decision')
        plt.subplots_adjust(hspace = 0.8)
    
    #Setup for decision_line graph
    if decision_line == True:
        decision_bound = np.zeros((time_steps-1, simulations))
    
    if runtime == True:
        start_time = time.time()
        
    #The exercise rule
    #The jth row tells me the exercise rule corresponding to the jth price set.
    exercise_rule = np.array([[time_steps for i in range(paths)] for j in range(simulations)])
    
    if option_type == "put":
        decision_steps = list(range(1, time_steps))
    elif option_type == "call":
        #The decision time will only matter just before dividend payment
        #decision_steps = list(range(1, time_steps))
        decision_steps = np.sort(time_steps - decision_steps)
        #print(decision_steps)
    
    warning_count = 0
    j = 0
    while j < simulations:
    #For each time step, update the exercise rule
        
        for i in decision_steps:
            #print("Progress: " + str(100*(i*simulations + j)/(time_steps*simulations)) + "%")
            #For each price set, update the exercise rule at the current time step.
            X_vals = []
            X_inds = []
            Y_vals = []

            #We only want to perform regression on paths where the exercise decision matters

            #Take the prices from the current candidate time, and filter out ones where the price is too high.
            price_set = prices[j][time_steps-1-i]
            
            #List of all indices where the decision matters
            if option_type == "put":
                X_inds = np.where(K - price_set > 0)
            elif option_type == "call":
                X_inds = np.where(price_set - K > 0)
                #print("Length of X_inds at simulation " + str(j) + " and decision step " + str(i) + " is: " + str(len(X_inds[0])))
            
            X_vals = price_set[X_inds]
            poi = np.array([prices[j][exercise_rule[j][k]-1][k] for k in range(paths)])
            #print("Number of nonpositive prices of interest: "+ str(len(np.where(poi <= 0)[0])))
            
            if option_type == "put":
                Y_vals = np.exp(-r*T*(exercise_rule[j][X_inds] - (time_steps-i))/time_steps)*np.maximum(K - poi[X_inds],0)
            elif option_type == "call":
                Y_vals = np.exp(-r*T*(exercise_rule[j][X_inds] - (time_steps-i))/time_steps)*np.maximum(poi[X_inds] - K,0)
            
            time_delta = T*(exercise_rule[j][X_inds] - (time_steps - i))/time_steps
            shocks = np.power(sigma,-1.0)*np.power(time_delta,-0.5)*(np.log(poi[X_inds]/X_vals) - (r - 0.5*(sigma**2))*time_delta)
            weights = (1/np.sqrt(2*np.pi))*np.exp(-0.5*(shocks**2))
            
            #weights = norm.pdf(shocks)
            
            if len(X_vals) > 0:
                #Perform regression on X_vals and Y_vals
                coefs = np.polynomial.polynomial.polyfit(X_vals, Y_vals, 2, w = weights)
                Z_vals = np.polynomial.polynomial.polyval(X_vals,coefs,2)

                #Plots for helpful visualization
                if graphs == True:
                    axs[i-1,j].scatter(X_vals, Y_vals)
                    axs[i-1,j].scatter(X_vals,Z_vals)
                    axs[i-1,j].scatter(X_vals, Z_vals)
                    if option_type == "put":
                        axs[i-1,j].scatter(X_vals,K - X_vals)
                    elif option_type == "call":
                        axs[i-1,j].scatter(X_vals,X_vals - K)
                
                #Update the exercise rule
                if len(np.where(Z_vals - (K - X_vals) <= 0)[0]) > 0 and option_type == "put":
                    ioi = np.where(Z_vals - (K - X_vals) <= 0)[0]
                    exercise_rule[j][X_inds[0][ioi]] = time_steps - i
                    if decision_line == True:
                        #Update the decision line with the highest price at which early exercise is optimal
                        decision_bound[time_steps - i - 1, j] = max(X_vals[ioi])
                        
                elif len(np.where(Z_vals - (X_vals - K) <= 0)[0]) > 0 and option_type == "call":
                    ioi = np.where(Z_vals - (X_vals - K) <= 0)[0]
                    exercise_rule[j][X_inds[0][ioi]] = time_steps - i
                    if decision_line == True:
                        #Update the decision line with the highest price at which early exercise is optimal
                        decision_bound[time_steps - i - 1, j] = max(X_vals[ioi])
                        
                else:
                    #If we reach this point, then there is no path where early exercise is optimal.
                    #decision_bound.extend([decision_bound[-1] for i in range(time_steps - 1 - len(decision_bound))])
                    if early_termination == True:
                        #print("Reached a state of early termination in simulation " + str(j) + " at time " + str(T*(time_steps - i)/time_steps))
                        break
                        
            else:
                #warning_count = warning_count + 1
                if early_termination == True:
                        #print("Reached a state of early termination in simulation " + str(j) + " at time " + str(T*(time_steps - i)/time_steps))
                        break
                #print("Warning " + str(warning_count) + "! X_vals was empty on iteration (i,j) = (" + str(i) + "," + str(j) + ")")
                #clear_output(wait=True)
        j = j + 1
    
    ##################
    #Compute the price
    
    #There is some dimension mismatch here, so let's check that
    #print("The shape of prices is " + str(np.shape(prices)))
    
    poi = np.array([[prices[i][exercise_rule[i][j] - 1][j] for j in range(paths)] for i in range(simulations)])
    #print("Deduced exercise rule matrix: " + str(exercise_rule))
    #print("Stock prices at exercise: " + str(poi))
    if option_type == "put":
        payoffs = np.exp(-r*T*((exercise_rule)/time_steps))*np.maximum(K - poi, 0)
    elif option_type == "call":
        payoffs = np.exp(-r*T*((exercise_rule)/time_steps))*np.maximum(poi - K, 0)
    
    ##########################
    #Display the decision line
    if decision_line == True:
        decision_fig, decision_axs = plt.subplots()
        decision_x = np.array([T*(i)/time_steps for i in range(1,time_steps)])
        decision_y = np.mean(decision_bound, axis = 1)
        decision_axs.plot(decision_x, decision_y)
        decision_axs.set_title("Decision Boundary For Option")
    
    ########################################################
    #Compute the probability that the option is in the money
    if Prob_ITM == True:
        sorted_payoffs = np.sort(payoffs.flatten())
        payoffs_cdf = np.array([len(np.where(sorted_payoffs <= sorted_payoffs[i])[0])/len(sorted_payoffs) for i in range(len(sorted_payoffs))])

        Prob_fig, Prob_axs = plt.subplots()
        Prob_axs.plot(sorted_payoffs, payoffs_cdf)
        Prob_axs.set_title("CDF of Option Payoff at Expiration")
        Prob_axs.set_ylim([0,1.05])
        
        option_price = np.mean(payoffs)
        print("The probability of the option expiring in the money is " + str(np.round(100*(1 - payoffs_cdf[0]),2)) + "%.")
        print("The probability of the option expiring for a profit is " + str(np.round(100*(1 - len(np.where(sorted_payoffs < option_price)[0])/len(sorted_payoffs)),2)) + "%.")
        print("The probability of the option expiring for a 50% gain is " + str(np.round(100*(1 - len(np.where(sorted_payoffs < 1.5*option_price)[0])/len(sorted_payoffs)),2)) + "%.")
    
        #Computation of the probability the option is ever in ITM
        max_prices = np.amax(prices, axis = 1).flatten()
        if option_type == "put":
            Prob_ITM_any = np.round(100*len(np.where(max_prices < K)[0])/len(max_prices),2)
        elif option_type == "call":
            Prob_ITM_any = np.round(100*len(np.where(max_prices > K)[0])/len(max_prices),2)
        print("The probability of the option ever being ITM is " + str(Prob_ITM_any) + "%.")
        
    ##########################
    #Show the time it took for the code to run
    if runtime == True:
        end_time = time.time()
        print("The code took " + str(np.round(end_time - start_time,2)) + " seconds to run!")
    
    #print("Payoffs: " + str(payoffs))
    return np.mean(payoffs) 

In the next few lines I give example cases to verify that the code is working as intended.

## Longstaff & Schwartz Test Example w/ Puts

Here I run a test of all the test cases considered from Table 1 of the Longstaff \& Schwartz paper. 

WARNING: The code will take some time to execute if a large number of paths and time steps is designated!

In [38]:
T = [1,2]
K = 40
r = 0.06
sigma = [0.2,0.4]
S_0 = [36,38,40,42,44]
time_steps = 50
paths = 50000
simulations = 1

table_of_values = []
total_tests = len(S_0)*len(sigma)*len(T)
i = 0
for s in S_0:
    for v in sigma:
        for t in T:
            a = LSM(t,K,r,v,s,time_steps,paths,simulations)
            b = np.array([[s,v,t,a]])
            if i == 0:
                table_of_values = b
            else:
                table_of_values = np.append(table_of_values,b,axis = 0)
            i = i + 1
            clear_output(wait=True)
            print("Completed computation of iteration " + str(i) + " of " + str(total_tests))

c = pd.DataFrame(table_of_values,columns=['S_0','sigma','T','Put price'])
clear_output(wait=True)
print(c)

     S_0  sigma    T  Put price
0   36.0    0.2  1.0   4.478375
1   36.0    0.2  2.0   4.808466
2   36.0    0.4  1.0   7.034325
3   36.0    0.4  2.0   8.415141
4   38.0    0.2  1.0   3.236993
5   38.0    0.2  2.0   3.717427
6   38.0    0.4  1.0   6.115529
7   38.0    0.4  2.0   7.581659
8   40.0    0.2  1.0   2.299441
9   40.0    0.2  2.0   2.880358
10  40.0    0.4  1.0   5.278922
11  40.0    0.4  2.0   6.870899
12  42.0    0.2  1.0   1.618901
13  42.0    0.2  2.0   2.219145
14  42.0    0.4  1.0   4.528538
15  42.0    0.4  2.0   6.188420
16  44.0    0.2  1.0   1.097758
17  44.0    0.2  2.0   1.680876
18  44.0    0.4  1.0   3.924207
19  44.0    0.4  2.0   5.583270


# Implied Volatility of American Options

The code to compute implied volatility is based off the secant method. Oddly enough, it seems the code for implied volatility is more predictable than the code to compute the option price.

In [41]:
def foo(mu, T, time_steps):
    #print(time_steps)
    val = np.zeros((1,time_steps,1))
    #print("Shape of val before: " + str(np.shape(val)))
    val[0,:,0] = mu*np.array([T*(i+1)/time_steps for i in range(time_steps)])
    #print("Shape of val after: " + str(np.shape(val)))
    return val

In [65]:
def impVolLSM(option_price, T, K , r, S_0, time_steps, paths, simulations, starting_sigmas, iterations):
    #Find the implied volatility of an American put as indicated by the LSM algorithm
    #To do the searching, I use the secant method
    
    #First, simulate the price paths we'll use
    prices = simulateBrownianMultiple(0, 1, T, time_steps, paths, simulations, antithetic=True)
    
    current_x = starting_sigmas[0]
    new_x = starting_sigmas[1]
    
    current_y = LSM(T, K, r, current_x, S_0, time_steps, paths, simulations, prices = S_0*np.exp(foo(r - 0.5*(current_x)**2, T, time_steps) + current_x*prices),set_prices = False) - option_price
    new_y = LSM(T, K, r, new_x, S_0, time_steps, paths, simulations, prices = S_0*np.exp(foo(r - 0.5*(new_x)**2, T, time_steps) + new_x*prices),set_prices = False) - option_price
    
    #print("Initial sigma values: (" + str(starting_sigmas[0]) + ", " + str(starting_sigmas[1]) + ")")
    #print("Initial error values: (" + str(new_y) + ", " + str(current_y) + ")")
    
    for i in range(iterations):
        
        previous_x = current_x
        current_x = new_x
        
        previous_y = current_y
        current_y = new_y

        #current_y = LSM_lightweight(T, K , r, current_x, S_0, prices)
        #previous_y = LSM_lightweight(T, K , r, previous_x, S_0, prices)
    
        new_x = current_x - current_y*(current_x - previous_x)/(current_y - previous_y)
        if np.isnan(new_x):
            new_x = current_x
            break
            
        new_y = LSM(T, K, r, new_x, S_0, time_steps, paths, simulations, prices = S_0*np.exp(foo(r - 0.5*(new_x)**2, T, time_steps) + new_x*prices),set_prices = False) - option_price
        
        #print("(sigma, error) = (" + str(new_x) + "," + str(new_y) + ").")
    
    #print("Implied volatility for strike price " + str(K) + ": " + str(np.round(new_x,5)))
    return new_x
    

## Longstaff and Schwartz Test Example w/ Puts

Given set values of $S_0$ and $T$ and simulated American put prices from Table 1 of Longstaff \& Schwartz, I compute the implied volatilities using secant method and Longstaff and Schwartz's algorithm. The values in the "imp vol" column should be close to those in the "sigma" column.

WARNING: This code despite optimization takes a really long time to run!

In [67]:
T = [1,2]
S_0 = [36,38,40,42,44]
sigma = [0.2,0.4]

option_prices = np.zeros((len(S_0),len(sigma),len(T)))

#S_0 = 36
option_prices[0,0,0] = 4.472
option_prices[0,0,1] = 4.821
option_prices[0,1,0] = 7.091
option_prices[0,1,1] = 8.488

#S_0 = 38
option_prices[1,0,0] = 3.244
option_prices[1,0,1] = 3.735
option_prices[1,1,0] = 6.139
option_prices[1,1,1] = 7.669

#S_0 = 40
option_prices[2,0,0] = 2.313
option_prices[2,0,1] = 2.879
option_prices[2,1,0] = 5.308
option_prices[2,1,1] = 6.921

#S_0 = 42
option_prices[3,0,0] = 1.617
option_prices[3,0,1] = 2.206
option_prices[3,1,0] = 4.588
option_prices[3,1,1] = 6.243

#S_0 = 44
option_prices[4,0,0] = 1.118
option_prices[4,0,1] = 1.675
option_prices[4,1,0] = 3.957
option_prices[4,1,1] = 5.622

K = 40
r = 0.06
time_steps = 50
paths = 50000
simulations = 1

table_of_values = []
total_tests = len(S_0)*len(sigma)*len(T)

i = 0
for s in range(len(S_0)):
    for v in range(len(sigma)):
        for t in range(len(T)):
            starting_sigmas = [0.9*sigma[v],1.1*sigma[v]]
            a = impVolLSM(option_prices[s,v,t],T[t],40,0.06,S_0[s],50,50000,1,starting_sigmas,10)
            b = np.array([[S_0[s],sigma[v],T[t],a]])
            if i == 0:
                table_of_values = b
            else:
                table_of_values = np.append(table_of_values,b,axis = 0)
            i = i + 1
            clear_output(wait=True)
            print("Completed computation of iteration " + str(i) + " of " + str(total_tests))

c = pd.DataFrame(table_of_values,columns=['S_0','sigma','T','imp vol'])
clear_output(wait=True)
print(c)



     S_0  sigma    T   imp vol
0   36.0    0.2  1.0  0.199238
1   36.0    0.2  2.0  0.201264
2   36.0    0.4  1.0  0.404459
3   36.0    0.4  2.0  0.402189
4   38.0    0.2  1.0  0.200053
5   38.0    0.2  2.0  0.200643
6   38.0    0.4  1.0  0.402693
7   38.0    0.4  2.0  0.403350
8   40.0    0.2  1.0  0.199375
9   40.0    0.2  2.0  0.200941
10  40.0    0.4  1.0  0.402557
11  40.0    0.4  2.0  0.403701
12  42.0    0.2  1.0  0.200420
13  42.0    0.2  2.0  0.200101
14  42.0    0.4  1.0  0.402987
15  42.0    0.4  2.0  0.402419
16  44.0    0.2  1.0  0.200846
17  44.0    0.2  2.0  0.199208
18  44.0    0.4  1.0  0.403599
19  44.0    0.4  2.0  0.401222
